# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TIRUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
startSwank 4011

In [ ]:
(ql:quickload :tirun)

In [ ]:
(leap:setup-amber-paths)

In [ ]:
(source "leaprc.protein.ff14SB")

In [ ]:
quickload :tirun-jupyter

# Start a new TIRUN calculation

In [ ]:
(tirun-jupyter:new-tirun)

In [ ]:
(defparameter *tiruns* (make-instance 'tirun:calculation))

# Import the sketch from a Chemdraw file

In [ ]:
(jupyter:png-file "ligand3.png" )

In [ ]:
(defparameter *sketch* (tirun:load-chem-draw-tirun "ligand3.cdxml"))

In [ ]:
(defparameter *cdagg* (chem:as-aggregate *sketch*))

-------------

In [ ]:
(setf sketch2d::*show-names* t)

In [ ]:
(define-symbol-macro loaded-ligands (tirun-jupyter::loaded-ligands tirun-jupyter::*app*))

In [ ]:
(setf chem:*verbose* nil)

In [ ]:
(setf loaded-ligands (tirun::assemble-ligands *tiruns* *sketch*))

In [ ]:
(tirun::summarize-stereochemistry loaded-ligands)

In [ ]:
show (sketch2d:sketch2d (first loaded-ligands))

In [ ]:
(chem:first-atom-with-name (first loaded-ligands) :c1)

In [ ]:
(chem:get-configuration (chem:first-atom-with-name (first loaded-ligands) :c1))

In [ ]:
(chem:get-configuration (chem:first-atom-with-name (first loaded-ligands) :c3))

(chem:set-configuration (chem:first-atom-with-name (first loaded-ligands) :C1) :right-handed)

(chem:set-configuration (chem:first-atom-with-name (first loaded-ligands) :C1) :right-handed)

In [ ]:
(chem:first-atom-with-name (first loaded-ligands) :c3)

In [ ]:
(tirun-jupyter:select-ligands)

In [ ]:
(core:getpid)

# Load the Thrombin PDB file

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *thrombin* (load-pdb "2zff_fixed.pdb" :ignore-missing-topology t))

In [ ]:
(build-unbuilt-hydrogens *thrombin*)

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(nglv::add-representation vthrombin "ball+stick" :selection "all")

In [ ]:
(remove-molecules *thrombin* 'solvent)

In [ ]:
(chem:setf-molecule-type (chem:content-with-name *thrombin* :|53U_3|) 'ligand)

In [ ]:
ligmol = (chem:content-with-name *thrombin* :|53U_3|)

In [ ]:
(chem:remove-molecule *thrombin* ligmol)

In [ ]:
show *thrombin*

In [ ]:
show ligmol

In [ ]:
(setf (tirun:receptors *tiruns*) (list *thrombin*))

# Load a template ligand

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")
(load-amber-params "gaff.dat")

In [ ]:
mol = (tirun::molecule (first (tirun::ligands *tiruns*)))

# Pose the new ligands onto the template ligand

In [ ]:
(tirun::pose-ligands-using-similarity *tiruns* ligmol)

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(defparameter *lig* (chem:make-aggregate))
(chem:add-matter *lig* ligmol)
(nglv::add-structure vthrombin (make-instance 'cando-structure :matter *lig*))

In [ ]:
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (tirun::molecule (first (tirun:ligands *tiruns*))))
(nglv::add-structure vthrombin (make-instance 'cando-structure :matter *moveable-agg*))

In [ ]:
vlm = (show ligmol)

In [ ]:
(third (tirun:ligands *tiruns*))

In [ ]:
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (tirun::molecule (third (tirun:ligands *tiruns*))))
(nglv::add-structure vlm (make-instance 'cando-structure :matter *moveable-agg*))

In [ ]:
show (tirun::molecule (first (tirun:ligands *tiruns*)))

In [ ]:
(tirun-jupyter:select-ligands)

In [ ]:
(show (tirun:layout-ligands *tiruns* :accessor 'tirun::molecule))

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(tirun-jupyter:lomap)

In [ ]:
(tirun:build-job-nodes *tiruns*)

In [ ]:
(tirun::connect-job-nodes *tiruns* :simple :connections 2 :stages 3 :windows 11)

In [ ]:
(tirun:jobs *tiruns*)

In [ ]:
(graph::save-graph *tiruns*)

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *work-list* (tirun::generate-jobs *tiruns*))

In [ ]:
*work-list*